In [78]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd

%matplotlib inline

In [79]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [80]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
label = train['label']

# dense_to_one_hot
Y = pd.get_dummies(label)
X = train.drop('label', axis=1) / 255 # normalize

In [82]:
X.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
def display_img(row):
    one_image = row.values.reshape(28,28)
    plt.axis('off')
    plt.imshow(one_image, cmap=cm.binary)

In [96]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(X, Y)

In [97]:
train_X.shape
train_Y.shape

(31500, 10)

In [98]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [99]:
x = tf.placeholder(tf.float32, shape=(None, train_X.shape[1]))
y_ = tf.placeholder(tf.float32, shape=(None, 10))

In [100]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

sess.run(tf.global_variables_initializer())

In [120]:
# y = x * W + b
y = tf.matmul(x, W) + b
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))


train = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# 評価関数

batch_size = 100
start = 0
indexes = np.random.permutation(train_X.index)
epoch = 0
for _ in range(10000):
    end = start + batch_size
    if end > train_X.shape[1]:
        indexes = np.random.permutation(train_X.index)
        end = train_X.shape[1]

        epoch +=1 

    
    batch_X = train_X.loc[indexes[start:end], :]
    batch_Y = train_Y.loc[indexes[start:end], :]
    
    train.run(feed_dict={x: batch_X.values, y_: batch_Y.values})
    start += batch_size


    
    

9993 0.856381


In [121]:

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(epoch, accuracy.eval(feed_dict={x: test_X.values, y_: test_Y.values}))

9993 0.856381
